1. Extracción y revisión del primer archivo, tdf_finishers_master.csv


In [1]:
import pandas as pd
import numpy as np

#Cargo el archivo tdf_finishers_master.csv y reviso
df_finishers = pd.read_csv('../data/raw/tdf_finishers_master.csv')

# display(df_finishers.head())
# display(df_finishers.info())
# display(df_finishers.nunique())

# display(df_finishers[df_finishers['Year'] == 1979])
#Se detecta un error en el archivo csv que impide cargar bien dos filas del año 1979.
#Localizamos la celda y arreglamos
mask_corrupta = df_finishers['Time'].str.contains('\r\n', na=False)

if mask_corrupta.any():
    idx = df_finishers[mask_corrupta].index[0]
    texto_sucio = df_finishers.at[idx, 'Time']
    equipo_desplazado = df_finishers.at[idx, 'Team']
    
    # Troceamos el texto para separar a Joop de Joaquim
    partes = texto_sucio.splitlines()
    
    # A. Arreglamos a Joop Zoetemelk (Fila actual)
    datos_joop = partes[0].split(',')
    df_finishers.at[idx, 'Time'] = datos_joop[0].replace('"', '')
    df_finishers.at[idx, 'Team'] = datos_joop[1]
    
    # B. Creamos a Joaquim Agostinho (Fila nueva)
    # No le ponemos Rank todavía, dejamos que el cumcount lo haga luego
    datos_joaquim = partes[1].split(',')
    fila_nueva = pd.DataFrame({
        'Year': [1979],
        'Rank': [3],
        'Rider': [datos_joaquim[2]],
        'Time': [datos_joaquim[3].replace('"', '').strip()],
        'Team': [equipo_desplazado]
    })
    
    # Insertamos la fila justo debajo de Joop para que el orden sea el de llegada
    df_finishers = pd.concat([df_finishers.iloc[:idx+1], fila_nueva, df_finishers.iloc[idx+1:]]).reset_index(drop=True)

    display(df_finishers[df_finishers['Year'] == 1979])


,Year,Rank,Rider,Time,Team
3747,1979,1,Bernard Hinault (FRA),"103h 06' 50""",Renault–Gitane
3748,1979,2,Joop Zoetemelk (NED),+ 13' 07,Miko–Mercier–Vivagel
3749,1979,3,Joaquim Agostinho (POR),+ 26' 53,Flandria–Ça va seul
3750,1979,4,Hennie Kuiper (NED),"+ 28' 02""",Peugeot–Esso–Michelin
3751,1979,5,Jean-René Bernaudeau (FRA),"+ 32' 43""",Renault–Gitane
...,...,...,...,...,...
3831,1979,85,José De Cauwer (BEL),"+ 3h 20' 28""",Peugeot–Esso–Michelin
3832,1979,86,Eulalio García (ESP),"+ 3h 27' 14""",Teka
3833,1979,87,Serge Beucherie (FRA),"+ 3h 38' 54""",Fiat
3834,1979,88,Alex Van Linden (BEL),"+ 3h 49' 57""",Bianchi–Faema


2. Limpieza de la columna Rank para convertirla a int


In [2]:
#Intento convertir la columna 'Rank' a int
#df_finishers['Rank'] = df_finishers['Rank'].astype(int)
#Salta error ya que hay caracteres no numericos que o se pueden convertir
#Muestro todos los caracteres unicos de la columna 'Rank' para identificar cuales son los que no se pueden convertir a int
is_not_numeric = ~df_finishers['Rank'].astype(str).str.isnumeric()
unique_non_numeric = df_finishers.loc[is_not_numeric, 'Rank']
# print("Valores no numéricos en 'Rank':")
# print(unique_non_numeric.unique())
print(f"Conteo del total de fuilas con valores no numéricos en 'Rank': {is_not_numeric.sum()}")
#Compruebo que hay 34 valores no numericos que habría que convertir a numérico. DSQ

# Para arreglarlo, recalculamos el Rank basado en el orden de las filas para cada año
# Usamos cumcount() + 1 para generar una secuencia 1, 2, 3... por cada grupo de 'Year', luego convierto a int la columna entera
df_finishers['Rank'] = df_finishers.groupby('Year').cumcount() + 1
df_finishers['Rank'] = df_finishers['Rank'].astype(int)


# Verifico algunos años para asegurarme que el Rank se ha recalculado correctamente
display(df_finishers[df_finishers['Year'] == 1999].head())
# display(df_finishers[df_finishers['Year'] == 2000].head())
# display(df_finishers[df_finishers['Year'] == 2001].head())



Conteo del total de fuilas con valores no numéricos en 'Rank': 34


,Year,Rank,Rider,Time,Team
6255,1999,1,Lance Armstrong (USA)[a],"91h 32' 16""",U.S. Postal Service
6256,1999,2,Alex Zülle (SUI),"+ 7' 37""",Banesto
6257,1999,3,Fernando Escartín (ESP),"+ 10' 26""",Kelme–Costa Blanca
6258,1999,4,Laurent Dufaux (SUI),"+ 14' 43""",Saeco Macchine per Caffè–Cannondale
6259,1999,5,Ángel Casero (ESP),"+ 15' 11""",Vitalicio Seguros


3. Limpieza de la columna Raider

In [3]:
# Eliminamos corchetes y su contenido
df_finishers['Rider'] = df_finishers['Rider'].str.replace(r'\[.*?\]', '', regex=True)

# Eliminamos espacios raros (\xa0, saltos de línea, etc.) y los convertimos en un espacio normal
df_finishers['Rider'] = df_finishers['Rider'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Usamos (.*) para el nombre para que capture todo hasta el ÚLTIMO paréntesis
# Esto evita que nombres como "Jordi (Jorge) Fortià (ESP)" se rompan
df_finishers[['Rider_name', 'Country']] = df_finishers['Rider'].str.extract(r'^(.*)\s\((.*)\)$')

# Si no encontró paréntesis, el nombre está en la columna original 'Rider'. Pasa con dos rusos, se añade el pais automáticamente.
df_finishers.loc[df_finishers['Rider_name'].isnull(), 'Rider_name'] = df_finishers['Rider']
df_finishers.loc[df_finishers['Country'].isnull(), 'Country'] = 'RUS'

#Limpieza final de espacios
df_finishers['Rider_name'] = df_finishers['Rider_name'].str.strip()
df_finishers['Country'] = df_finishers['Country'].str.strip()

# Verificación: muestra los que antes tenían corchetes (como el Rank 4 de 1903)
# display(df_finishers[df_finishers['Year'] == 1903].head(5))

#Elimino columna Rider
df_finishers.drop(columns=['Rider'], inplace = True)

display(df_finishers)

,Year,Rank,Time,Team,Rider_name,Country
0,1903,1,"94h 33' 14""",La Française,Maurice Garin,FRA
1,1903,2,"+ 2h 59' 21""",La Française,Lucien Pothier,FRA
2,1903,3,"+ 4h 29' 24""",La Française,Fernand Augereau,FRA
3,1903,4,"+ 4h 39' 30""",La Française,Rodolfo Muller,ITA
4,1903,5,"+ 4h 58' 44""",La Française,Jean Fischer,FRA
...,...,...,...,...,...,...
10340,2025,156,"+ 5h 44' 11""""",Intermarché - Wanty,Roel van Sintmaartensdijk,NLD
10341,2025,157,"+ 5h 46' 00""""",Tudor Pro Cycling Team,Fabian Lienhard,SUI
10342,2025,158,"+ 5h 48' 25""""",Red Bull - BORA - hansgrohe,Jordi Meeus,BEL
10343,2025,159,"+ 5h 51' 25""""",Lidl - Trek,Edward Theuns,BEL


4. Comprobamos y limpiamos la columna 'Country'

In [4]:
#Compruebo en la columna 'Country' que todos los nombres de los paises estén normalzados
# print(df_finishers['Country'].unique())
#Compruebo que hay datos no unificados, como 'ITA' e 'Italia'
#Creo uin diccionario para unificar datos.
unificated_country = {'FRA': 'Francia', 'ITA': 'Italia', 'BEL': 'Belgica', 'GER': 'Alemania', 'LUX': 'Luxemburgo',
                    'SUI': 'Suiza', 'Italy': 'Italia', 'AUS': 'Australia', 'MON': 'Mónaco', 'ESP': 'España', 'NZL': 'Nueva Zelanda',
                    'AUT': 'Austria', 'NED': 'Países Bajos', 'ALG': 'Argelia', 'POL': 'Polonia', 'GBR': 'Reino Unido',
                    'FRG': 'Alemania', 'POR': 'Portugal', 'IRL': 'Irlanda', 'DEN': 'Dinamarca', 'SWE': 'Suecia',
                    'COL': 'Colombia', 'NOR': 'Noruega', 'IRE': 'Irlanda', 'USA': 'Estados Unidos', 'CAN': 'Canada',
                    'YUG': 'Yugoslavia', 'MEX': 'México', 'TCH': 'Checoslovaquia', 'URS': 'Urss', 'DDR': 'Alemania',
                    'UZB': 'Uzbekistán', 'UKR': 'Ucrania', 'BRA': 'Brasil', 'LIT': 'Lituania', 'RUS': 'Rusia',
                    'VEN': 'Venezuela', 'LAT': 'Letonia', 'SVK': 'Eslovaquia', 'FIN': 'Finlandia', 'EST': 'Estonia',
                    'KAZ': 'Kazajistán', 'CZE': 'República Checa', 'SPA': 'España', 'LTU': 'Lituania', 'HUN': 'Hungría',
                    'SAF': 'Sudáfrica', 'SLO': 'Eslovenia', 'CRO': 'Croacia', 'BLR': 'Bielorrusia', 'RSA': 'Sudáfrica',
                    'JPN': 'Japón', 'MDA': 'Moldavia', 'SVN': 'Eslovenia', 'CRC': 'Costa Rica', 'ARG': 'Argentina',
                    'UK': 'Reino Unido', 'CHN': 'China', 'ERI': 'Eritrea', 'SWI': 'Suiza', 'ETH': 'Etiopía', 
                    'ECU': 'Ecuador', 'ISR': 'Israel', 'NLD': 'Paises Bajos'}

#Recorro la columna sustituyendo los valores para unificar.
df_finishers['Country_normalized'] = df_finishers['Country'].map(unificated_country)

# display(df_finishers[['Country_normalized', 'Country']])
#Eliminamos la columna 'Country' y renombramos 'Country_normalized' a 'country'
df_finishers.drop(columns='Country', inplace=True)
df_finishers.rename (columns = {'Country_normalized': 'country'}, inplace = True)

display(df_finishers)


,Year,Rank,Time,Team,Rider_name,country
0,1903,1,"94h 33' 14""",La Française,Maurice Garin,Francia
1,1903,2,"+ 2h 59' 21""",La Française,Lucien Pothier,Francia
2,1903,3,"+ 4h 29' 24""",La Française,Fernand Augereau,Francia
3,1903,4,"+ 4h 39' 30""",La Française,Rodolfo Muller,Italia
4,1903,5,"+ 4h 58' 44""",La Française,Jean Fischer,Francia
...,...,...,...,...,...,...
10340,2025,156,"+ 5h 44' 11""""",Intermarché - Wanty,Roel van Sintmaartensdijk,Paises Bajos
10341,2025,157,"+ 5h 46' 00""""",Tudor Pro Cycling Team,Fabian Lienhard,Suiza
10342,2025,158,"+ 5h 48' 25""""",Red Bull - BORA - hansgrohe,Jordi Meeus,Belgica
10343,2025,159,"+ 5h 51' 25""""",Lidl - Trek,Edward Theuns,Belgica


5. Limpieza de la columna Time

In [5]:
# Compruebo si hay nulos en la columna Time antes de convertir a segundos
# print(f"Total de nulos en 'Time': {df_finishers['Time'].isnull().sum()}")
# # Como en los años 1905 - 1912 no había clasifiacion por puntos, compruebo los nulos por años
# nulos_por_año = df_finishers[df_finishers['Time'].isnull()]['Year'].value_counts().sort_index()
# print("Nulos en 'Time' por año:")       
# print(nulos_por_año)

# Convierto la columna Time a segundos, dejando los nulos como tales. Primero la función.
def time_to_seconds(time_str):
    if pd.isnull(time_str):
        return np.nan
    clean_time = time_str.replace('"', ' ').replace('+', '').replace('h', '').replace("'", "").replace("′", "").replace("″", "").split()
    if len(clean_time) == 3:
        hours, minutes, seconds = clean_time
        return int(hours) * 3600 + int(minutes) * 60 + int(seconds)
    elif len(clean_time) == 2:
        minutes, seconds = clean_time
        return int(minutes) * 60 + int(seconds)
    elif len(clean_time) == 1:
        return int(clean_time[0])
    else:
        return np.nan

# # Aplico la función a la columna Time para crear una nueva columna Time_seconds
df_finishers['Time_seconds'] = df_finishers['Time'].apply(time_to_seconds)
# display(df_finishers[['Time', 'Time_seconds']].head(20))

# Ahora, año a año, sumo los segundos del primero a cada uno de los otros corredores para que salga el total de segundos 
# que ha empleado cada corredor
df_finishers['winner_time_base'] = df_finishers.groupby('Year')['Time_seconds'].transform('first')
df_finishers['rider_total_time'] = np.where(
    df_finishers['Time_seconds'] == df_finishers['winner_time_base'],
    df_finishers['winner_time_base'],
    df_finishers['winner_time_base'] + df_finishers['Time_seconds']
)           

# #compruebo que el cálculo de los segundos totales es correcto
# display (df_finishers[df_finishers['Year'] == 2000].head(20))

# #Elimino columnas 'winner_time_base', 'Time_seconds' y 'Time' que ya no necesito
df_finishers.drop(columns=['winner_time_base', 'Time_seconds', 'Time'], inplace=True)
display(df_finishers.info())
display(df_finishers.head())
display(df_finishers)

<class 'pandas.DataFrame'>
RangeIndex: 10345 entries, 0 to 10344
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              10345 non-null  int64  
 1   Rank              10345 non-null  int64  
 2   Team              10042 non-null  str    
 3   Rider_name        10345 non-null  str    
 4   country           10345 non-null  str    
 5   rider_total_time  10073 non-null  float64
dtypes: float64(1), int64(2), str(3)
memory usage: 485.1 KB


None

,Year,Rank,Team,Rider_name,country,rider_total_time
0,1903,1,La Française,Maurice Garin,Francia,340394.0
1,1903,2,La Française,Lucien Pothier,Francia,351155.0
2,1903,3,La Française,Fernand Augereau,Francia,356558.0
3,1903,4,La Française,Rodolfo Muller,Italia,357164.0
4,1903,5,La Française,Jean Fischer,Francia,358318.0


,Year,Rank,Team,Rider_name,country,rider_total_time
0,1903,1,La Française,Maurice Garin,Francia,340394.0
1,1903,2,La Française,Lucien Pothier,Francia,351155.0
2,1903,3,La Française,Fernand Augereau,Francia,356558.0
3,1903,4,La Française,Rodolfo Muller,Italia,357164.0
4,1903,5,La Française,Jean Fischer,Francia,358318.0
...,...,...,...,...,...,...
10340,2025,156,Intermarché - Wanty,Roel van Sintmaartensdijk,Paises Bajos,294283.0
10341,2025,157,Tudor Pro Cycling Team,Fabian Lienhard,Suiza,294392.0
10342,2025,158,Red Bull - BORA - hansgrohe,Jordi Meeus,Belgica,294537.0
10343,2025,159,Lidl - Trek,Edward Theuns,Belgica,294717.0


6.Últimos cambios y guardado del dataset

In [6]:
#Cambiamos de orden las columnas
new_order = ['Year', 'Rank', 'Rider_name', 'country', 'Team', 'rider_total_time']
df_clean_finishers = df_finishers[new_order]
df_clean_finishers.columns = df_clean_finishers.columns.str.lower()
display(df_clean_finishers)

#Ya estaría la limpieza de este dataset. Guardo el dataset limpio en la carpeta processed para usarlo en el siguiente notebook con el nombre tdf_finishers_clean.csv
df_clean_finishers.to_csv('../data/processed/tdf_finishers_clean.csv', index=False)

,year,rank,rider_name,country,team,rider_total_time
0,1903,1,Maurice Garin,Francia,La Française,340394.0
1,1903,2,Lucien Pothier,Francia,La Française,351155.0
2,1903,3,Fernand Augereau,Francia,La Française,356558.0
3,1903,4,Rodolfo Muller,Italia,La Française,357164.0
4,1903,5,Jean Fischer,Francia,La Française,358318.0
...,...,...,...,...,...,...
10340,2025,156,Roel van Sintmaartensdijk,Paises Bajos,Intermarché - Wanty,294283.0
10341,2025,157,Fabian Lienhard,Suiza,Tudor Pro Cycling Team,294392.0
10342,2025,158,Jordi Meeus,Belgica,Red Bull - BORA - hansgrohe,294537.0
10343,2025,159,Edward Theuns,Belgica,Lidl - Trek,294717.0
